Learn more about AnimeGANv2 here: https://github.com/bryandlee/animegan2-pytorch

To start using this, run the cells with Ctrl+F9 or "Runtime > Run All"

For accelerated inference, you can use a GPU. Simply select "Runtime > Change runtime type" and select "GPU" in the "Hardware Acceleration" dropdown.

# SageMaker BYOC

In [ ]:
!aws ecr get-login-password --region us-west-2|docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.9.1-gpu-py38-cu111-ubuntu20.04

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-animegan2-pytorch

cd container

chmod +x byoc/train
chmod +x byoc/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-cn-north-1}

uri_suffix="amazonaws.com"
if [ "$region" == "cn-north-1" ] || [ "$region" == "cn-northwest-1" ]; then
    uri_suffix="amazonaws.com.cn"
fi    

fullname="${account}.dkr.ecr.${region}.${uri_suffix}/${algorithm_name}:byoc"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
# Define IAM role
import boto3

import os
from sagemaker import get_execution_role
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()
role = get_execution_role()

In [ ]:
from sagemaker.model import Model
from sagemaker.serializers import IdentitySerializer

img_serializer = IdentitySerializer(content_type='image/png')

endpoint_name = 'sagemaker-animegan2-pytorch' #SET_YOUR_ENDPOINT_NAME
image = '169088282855.dkr.ecr.us-west-2.amazonaws.com/sagemaker-animegan2-pytorch:byoc' #SET_YOUR_IMAGE_URI which build above

model = Model(image, role=role)
endpoint = model.deploy(1, "ml.g4dn.xlarge", endpoint_name=endpoint_name, serializer=img_serializer)

In [ ]:
from sagemaker.predictor import Predictor
img_serializer = IdentitySerializer(content_type='application/x-image')

predictor = Predictor(endpoint_name, serializer=img_serializer)

-----

In [ ]:
file_name = 'WangShishuai.png'
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

In [ ]:
from io import BytesIO
from PIL import Image

result = predictor.predict(payload)

bytes_stream = BytesIO(result)
result_image = Image.open(bytes_stream)

In [ ]:
display(result_image)

In [ ]:
predictor.delete_endpoint()